<a href="https://colab.research.google.com/github/eshmaneva/DS-Practical/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Preinstalling the necessary libraries
#Certain versions are required to avoid compatibility issues 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy==1.19.5
!pip install tensorflow==2.7.0
!pip install transformers==4.7.0
!pip install sacremoses==0.0.45

     |████████████████████████████████| 14.8 MB 8.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 489.6 MB 23 kB/s 
     |████████████████████████████████| 1.3 MB 47.5 MB/s 
     |████████████████████████████████| 463 kB 66.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:
      Successfully uninstalled tensorflow-2.8.0


     |████████████████████████████████| 2.5 MB 7.7 MB/s 
     |████████████████████████████████| 3.3 MB 42.3 MB/s 
     |████████████████████████████████| 880 kB 55.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2687b5a15b10b3bab807eeea9fa6cf643bc3da72f9f54ee678ce3d51c6bd3599
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
     |████████████████████████████████| 895 kB 9.9 MB/s 
  Attempting uninstall: sacremoses
    Found existing installation: sacremoses 0.0.53
    Uninstalling sacremoses-0.0.53:
      Successfully uninstalled sacremoses-0.0.53


In [ ]:
#Importing necessary classes for classification and summarizaton
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

import pandas as pd
import numpy as np
import nltk
import re

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from six import viewitems
#Importing methods for splitting and shuffling data (as dataset contains no pre-trained data)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Reviews.csv", engine="python", error_bad_lines=False)
df.drop(columns=['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator','HelpfulnessDenominator', 'Time'],axis=1,inplace=True)
print("Before",len(df))
df = df.dropna()
print("Data size:",len(df))
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


Before 568454
Data size: 568427


,Score,Summary,Text
0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,2,Cough Medicine,If you are looking for the secret ingredient i...
4,5,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
#Checking the avaliable GPUs (not necessary, made as a test of the system)

#num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
#print("Num GPUs Available: ", num_gpus_available)
#assert num_gpus_available > 0

In [ ]:
#Used dataset: Amazon product reviews / Mobile Electronics
'''
ds = tfds.load('/content/drive/MyDrive/Reviews.csv', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
#print(ds)

'''

"\nds = tfds.load('/content/drive/MyDrive/Reviews.csv', split='train', shuffle_files=True)\nassert isinstance(ds, tf.data.Dataset)\n#print(ds)\n\n"

In [ ]:
'''
#Setting the dataset as a frame (trasforming it from tensor)
df = tfds.as_dataframe(df)
#Preview of the data
df.head()
'''

'\n#Setting the dataset as a frame (trasforming it from tensor)\ndf = tfds.as_dataframe(df)\n#Preview of the data\ndf.head()\n'

In [ ]:
#Classifying the data into two classes: positive and negative based on their star rating
df["Sentiment"] = df["Score"].apply(lambda score: "positive" if score >= 3 else "negative")
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})

In [ ]:
#df['short_review'] = df['Text'].str.decode("utf-8")

In [ ]:
df = df[["Text", "Sentiment"]]

In [ ]:
'''
#Dropping last n rows using drop
n = 54975
df.drop(df.tail(n).index,
        inplace = True)
'''

'\n#Dropping last n rows using drop\nn = 54975\ndf.drop(df.tail(n).index,\n        inplace = True)\n'

In [ ]:
df=df.loc[1:5000]

df.dropna()
print("Data size:",len(df))


Data size: 5000


In [ ]:
df.head()

,Text,Sentiment
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,1
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price. There was a wid...,1
5,I got a wild hair for taffy and ordered this f...,1


In [ ]:
#To check how big is the dataset / num of rows
#index = df.index
#number_of_rows = len(index)
#print(number_of_rows)

#Printing the beginning part to see if the data is read correctly
#df.head()

#Printing the beginning part to see if the data is read correctly
#df.tail()

In [ ]:
#Testing the labels
reviews = df['Text'].values.tolist()
labels = df['Sentiment'].tolist()
#print(reviews[:2])
#print(labels[:2])

In [ ]:
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)
#training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=0.33, random_state=42, stratify=labels)


In [ ]:
#Preprocessing the data using DistilBert for punctuation splitting and wordpieces
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
tokenizer([training_sentences[0]], truncation=True,
                            padding=True, max_length=128)

{'input_ids': [[101, 1045, 4156, 2122, 2138, 1997, 1996, 2659, 5474, 1010, 1998, 2027, 2020, 5262, 2659, 1999, 5474, 1012, 2174, 1010, 2116, 1010, 2116, 1997, 1996, 11772, 1999, 1996, 4524, 2020, 6719, 14309, 1999, 3514, 1012, 1045, 2031, 2196, 8567, 2023, 3291, 2007, 2060, 14557, 9090, 9639, 1012, 2009, 2001, 2200, 14477, 29519, 6026, 1010, 1998, 2040, 4122, 2035, 2008, 3514, 1029, 1045, 2097, 2196, 4965, 2122, 2153, 1010, 1998, 1045, 2052, 2025, 16755, 2068, 2000, 2500, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:

train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

print(val_dataset)

<TensorSliceDataset shapes: ({input_ids: (512,), attention_mask: (512,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>


In [ ]:
#tbd
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
callbacks=tf.keras.callbacks.EarlyStopping(
    monitor='accuracy', 
    min_delta=0.0001,
    patience=3,
    mode='auto',
    verbose=2,
    baseline=None
)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=2,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16),callbacks=callbacks)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [ ]:
'''
import matplotlib.pyplot as plt

plt.title('Loss curves')
plt.plot(model.train_loss_history, '-', label='train')
plt.plot(model.val_loss_history, '-', label='val')
plt.legend(loc='lower right')
plt.xlabel('Iteration')
plt.show()

'''

"\nimport matplotlib.pyplot as plt\n\nplt.title('Loss curves')\nplt.plot(model.train_loss_history, '-', label='train')\nplt.plot(model.val_loss_history, '-', label='val')\nplt.legend(loc='lower right')\nplt.xlabel('Iteration')\nplt.show()\n\n"

In [ ]:
model.save_pretrained("./sentiment")

In [ ]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("./sentiment")

Some layers from the model checkpoint at ./sentiment were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./sentiment and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import pandas as pd
#Testing a model with a user-written input

#df = pd.DataFrame({'Text': ["This is a not a good product. I hate it", "This product is okay", "I hate you", 'I think this apple is good', "This toast is terrible"]})
#test_sentence = "This is a not a good product. I hate it"

df1 = pd.read_csv("/content/drive/MyDrive/summary_test.csv", index_col=0, engine="python", error_bad_lines=False)
df1=df1.loc[0:50]
selected_columns = df1[["Summary","Text","Generated_summary"]]
df = selected_columns.copy()
df = df.dropna()


df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,Summary,Text,Generated_summary
0,Dessert to Them. But is it Nutritional Enough?,My twin 10-month old babies treat this one as ...,Sweet Potato
1,Repeated deliveries of broken jars. Great pro...,I liked getting this one for my twin 10-month ...,I like this one because it's vegetarian
2,"Good flavor, too runny though","Baby loved this one, but as he has progressed ...",I love this one!
3,12% Protein and 50% Vitamin A,Earth's Best Turkey Vegetable Dinner hits the ...,Delicious!
4,Baby Lilly says 2 thumbs up!,My little girl can't get enough of this! I lik...,Delicious!


In [ ]:
for i in range(0, len(df)):
  df['Sentiment_text'] = i 
  df['Sentiment_summary'] = i

In [ ]:
for i in range(0,len(df)):
  
  predict_input_text = tokenizer.encode(df['Text'][i],
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
  tf_output_text = loaded_model.predict(predict_input_text)[0]
  tf_prediction_text = tf.nn.softmax(tf_output_text, axis=1)
  labels = ['Negative','Positive']
  label_text = tf.argmax(tf_prediction_text, axis=1)
  label_text = label_text.numpy()
  df["Sentiment_text"][i] = (labels[label_text[0]])

  
#df = df.append(data, columns = "Sentiment")
#print(df['Text'], df['Sentiment_text'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
for i in range(0, len(df)):
  
  predict_input_sum = tokenizer.encode(df['Generated_summary'][i],
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
  tf_output_sum = loaded_model.predict(predict_input_sum)[0]
  tf_prediction_sum = tf.nn.softmax(tf_output_sum, axis=1)
  #labels = ['Negative','Positive']
  label_sum = tf.argmax(tf_prediction_sum, axis=1)
  label_sum = label_sum.numpy()
  df["Sentiment_summary"][i] = (labels[label_sum[0]])


#print(df['Generated_summary'], df['Sentiment_summary'])
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Summary,Text,Generated_summary,Sentiment_text,Sentiment_summary
0,Dessert to Them. But is it Nutritional Enough?,My twin 10-month old babies treat this one as ...,Sweet Potato,Positive,Positive
1,Repeated deliveries of broken jars. Great pro...,I liked getting this one for my twin 10-month ...,I like this one because it's vegetarian,Positive,Positive
2,"Good flavor, too runny though","Baby loved this one, but as he has progressed ...",I love this one!,Positive,Positive
3,12% Protein and 50% Vitamin A,Earth's Best Turkey Vegetable Dinner hits the ...,Delicious!,Positive,Positive
4,Baby Lilly says 2 thumbs up!,My little girl can't get enough of this! I lik...,Delicious!,Positive,Positive


In [ ]:
tag = 0
for i in range(0, len(df)):
  if (df['Sentiment_text'][i] != df['Sentiment_summary'][i]):
    tag = tag + 1

Error = tag/len(df)
print(Error)

0.12
